In [253]:
import pandas as pd
import requests
import sqlite3
import time

from config import API_KEY

In [280]:
params = {"key": API_KEY}
core_url = "https://api.sportsdata.io/api/mlb/fantasy/json/"
odds_url = "https://api.sportsdata.io/api/mlb/odds/json/"

def get_stadiums():
    r = requests.get(odds_url + "Stadiums", params=params)
    if r.status_code == 200:
        return pd.DataFrame(r.json())
    else:
        r.raise_for_status()

def get_players():
    r = requests.get(core_url + "Players", params=params)
    r2 = requests.get(core_url + "FreeAgents", params=params)
    if r.status_code == 200:
        return pd.concat(
            [pd.DataFrame(r.json()),
             pd.DataFrame(r2.json())
            ]
        )
    else:
        r.raise_for_status()

def get_teams():
    r = requests.get(core_url + "Teams", params=params)
    if r.status_code == 200:
        return pd.DataFrame(r.json())
    else:
        r.raise_for_status()

def get_games(date):
    r = requests.get(odds_url + f"GamesByDate/{date}", params=params)
    if r.status_code == 200:
        return pd.DataFrame(r.json())
    else:
        r.raise_for_status()

def get_player_stats(date):
    r = requests.get(core_url + f"PlayerGameStatsByDate/{date}", params=params)
    if r.status_code == 200:
        return pd.DataFrame(r.json())
    else:
        r.raise_for_status()

In [298]:
def insert_stadiums():
    data = get_stadiums()
    conn = sqlite3.connect("mlb.db")
    with conn:
        qmarks = str(tuple('?') * 22).replace("'", '')
        conn.executemany(f"INSERT OR REPLACE INTO Stadiums VALUES {qmarks}", list(data.itertuples(index=False, name=None)))
    conn.close()
    
def insert_teams():
    data = get_teams()
    conn = sqlite3.connect("mlb.db")
    with conn:
        qmarks = str(tuple('?') * 15).replace("'", '')
        conn.executemany(f"INSERT OR REPLACE INTO Teams VALUES {qmarks}", list(data.itertuples(index=False, name=None)))
    conn.close()
    
def insert_players():
    data = get_players()
    conn = sqlite3.connect("mlb.db")
    with conn:
        qmarks = str(tuple('?') * 23).replace("'", '')
        conn.executemany(f"INSERT OR REPLACE INTO Players VALUES {qmarks}", list(data.itertuples(index=False, name=None)))
    conn.close()

In [313]:
def insert_games(season):
    """
    For a given season(year), generates list of possible game dates for that season,
    fetches games for each date, and if there is data, inserts into the database
    """
    # Regular seasons run from at most mid-march to mid-november
    season_dates = pd.date_range(f"{season}-03-15", f"{season}-11-15", freq="d")
    conn = sqlite3.connect("mlb.db")
    for date in season_dates:
        iso_date = date.strftime("%Y-%m-%d")
        data = get_games(iso_date)
        # If dataframe is non-empty, continue
        if len(data) > 0:
            print(f"Fetching games for {date}")
            data = data.drop(columns="Innings")
            with conn:
                qmarks = str(tuple('?') * 34).replace("'", '')
                conn.executemany(f"INSERT OR REPLACE INTO Games VALUES {qmarks}", list(data.itertuples(index=False, name=None)))
        # Wait a half second before another API call
        time.sleep(0.5)
        
    conn.close()

In [314]:
seasons = range(2010, 2016)
for year in seasons:
    insert_games(year)

Fetching games for 2010-04-04 00:00:00
Fetching games for 2010-04-05 00:00:00
Fetching games for 2010-04-06 00:00:00
Fetching games for 2010-04-07 00:00:00
Fetching games for 2010-04-08 00:00:00
Fetching games for 2010-04-09 00:00:00
Fetching games for 2010-04-10 00:00:00
Fetching games for 2010-04-11 00:00:00
Fetching games for 2010-04-12 00:00:00
Fetching games for 2010-04-13 00:00:00
Fetching games for 2010-04-14 00:00:00
Fetching games for 2010-04-15 00:00:00
Fetching games for 2010-04-16 00:00:00
Fetching games for 2010-04-17 00:00:00
Fetching games for 2010-04-18 00:00:00
Fetching games for 2010-04-19 00:00:00
Fetching games for 2010-04-20 00:00:00
Fetching games for 2010-04-21 00:00:00
Fetching games for 2010-04-22 00:00:00
Fetching games for 2010-04-23 00:00:00
Fetching games for 2010-04-24 00:00:00
Fetching games for 2010-04-25 00:00:00
Fetching games for 2010-04-26 00:00:00
Fetching games for 2010-04-27 00:00:00
Fetching games for 2010-04-28 00:00:00
Fetching games for 2010-0

Fetching games for 2011-04-10 00:00:00
Fetching games for 2011-04-11 00:00:00
Fetching games for 2011-04-12 00:00:00
Fetching games for 2011-04-13 00:00:00
Fetching games for 2011-04-14 00:00:00
Fetching games for 2011-04-15 00:00:00
Fetching games for 2011-04-16 00:00:00
Fetching games for 2011-04-17 00:00:00
Fetching games for 2011-04-18 00:00:00
Fetching games for 2011-04-19 00:00:00
Fetching games for 2011-04-20 00:00:00
Fetching games for 2011-04-21 00:00:00
Fetching games for 2011-04-22 00:00:00
Fetching games for 2011-04-23 00:00:00
Fetching games for 2011-04-24 00:00:00
Fetching games for 2011-04-25 00:00:00
Fetching games for 2011-04-26 00:00:00
Fetching games for 2011-04-27 00:00:00
Fetching games for 2011-04-28 00:00:00
Fetching games for 2011-04-29 00:00:00
Fetching games for 2011-04-30 00:00:00
Fetching games for 2011-05-01 00:00:00
Fetching games for 2011-05-02 00:00:00
Fetching games for 2011-05-03 00:00:00
Fetching games for 2011-05-04 00:00:00
Fetching games for 2011-0

Fetching games for 2012-04-19 00:00:00
Fetching games for 2012-04-20 00:00:00
Fetching games for 2012-04-21 00:00:00
Fetching games for 2012-04-22 00:00:00
Fetching games for 2012-04-23 00:00:00
Fetching games for 2012-04-24 00:00:00
Fetching games for 2012-04-25 00:00:00
Fetching games for 2012-04-26 00:00:00
Fetching games for 2012-04-27 00:00:00
Fetching games for 2012-04-28 00:00:00
Fetching games for 2012-04-29 00:00:00
Fetching games for 2012-04-30 00:00:00
Fetching games for 2012-05-01 00:00:00
Fetching games for 2012-05-02 00:00:00
Fetching games for 2012-05-03 00:00:00
Fetching games for 2012-05-04 00:00:00
Fetching games for 2012-05-05 00:00:00
Fetching games for 2012-05-06 00:00:00
Fetching games for 2012-05-07 00:00:00
Fetching games for 2012-05-08 00:00:00
Fetching games for 2012-05-09 00:00:00
Fetching games for 2012-05-10 00:00:00
Fetching games for 2012-05-11 00:00:00
Fetching games for 2012-05-12 00:00:00
Fetching games for 2012-05-13 00:00:00
Fetching games for 2012-0

Fetching games for 2013-04-26 00:00:00
Fetching games for 2013-04-27 00:00:00
Fetching games for 2013-04-28 00:00:00
Fetching games for 2013-04-29 00:00:00
Fetching games for 2013-04-30 00:00:00
Fetching games for 2013-05-01 00:00:00
Fetching games for 2013-05-02 00:00:00
Fetching games for 2013-05-03 00:00:00
Fetching games for 2013-05-04 00:00:00
Fetching games for 2013-05-05 00:00:00
Fetching games for 2013-05-06 00:00:00
Fetching games for 2013-05-07 00:00:00
Fetching games for 2013-05-08 00:00:00
Fetching games for 2013-05-09 00:00:00
Fetching games for 2013-05-10 00:00:00
Fetching games for 2013-05-11 00:00:00
Fetching games for 2013-05-12 00:00:00
Fetching games for 2013-05-13 00:00:00
Fetching games for 2013-05-14 00:00:00
Fetching games for 2013-05-15 00:00:00
Fetching games for 2013-05-16 00:00:00
Fetching games for 2013-05-17 00:00:00
Fetching games for 2013-05-18 00:00:00
Fetching games for 2013-05-19 00:00:00
Fetching games for 2013-05-20 00:00:00
Fetching games for 2013-0

Fetching games for 2014-04-30 00:00:00
Fetching games for 2014-05-01 00:00:00
Fetching games for 2014-05-02 00:00:00
Fetching games for 2014-05-03 00:00:00
Fetching games for 2014-05-04 00:00:00
Fetching games for 2014-05-05 00:00:00
Fetching games for 2014-05-06 00:00:00
Fetching games for 2014-05-07 00:00:00
Fetching games for 2014-05-08 00:00:00
Fetching games for 2014-05-09 00:00:00
Fetching games for 2014-05-10 00:00:00
Fetching games for 2014-05-11 00:00:00
Fetching games for 2014-05-12 00:00:00
Fetching games for 2014-05-13 00:00:00
Fetching games for 2014-05-14 00:00:00
Fetching games for 2014-05-15 00:00:00
Fetching games for 2014-05-16 00:00:00
Fetching games for 2014-05-17 00:00:00
Fetching games for 2014-05-18 00:00:00
Fetching games for 2014-05-19 00:00:00
Fetching games for 2014-05-20 00:00:00
Fetching games for 2014-05-21 00:00:00
Fetching games for 2014-05-22 00:00:00
Fetching games for 2014-05-23 00:00:00
Fetching games for 2014-05-24 00:00:00
Fetching games for 2014-0

Fetching games for 2015-05-16 00:00:00
Fetching games for 2015-05-17 00:00:00
Fetching games for 2015-05-18 00:00:00
Fetching games for 2015-05-19 00:00:00
Fetching games for 2015-05-20 00:00:00
Fetching games for 2015-05-21 00:00:00
Fetching games for 2015-05-22 00:00:00
Fetching games for 2015-05-23 00:00:00
Fetching games for 2015-05-24 00:00:00
Fetching games for 2015-05-25 00:00:00
Fetching games for 2015-05-26 00:00:00
Fetching games for 2015-05-27 00:00:00
Fetching games for 2015-05-28 00:00:00
Fetching games for 2015-05-29 00:00:00
Fetching games for 2015-05-30 00:00:00
Fetching games for 2015-05-31 00:00:00
Fetching games for 2015-06-01 00:00:00
Fetching games for 2015-06-02 00:00:00
Fetching games for 2015-06-03 00:00:00
Fetching games for 2015-06-04 00:00:00
Fetching games for 2015-06-05 00:00:00
Fetching games for 2015-06-06 00:00:00
Fetching games for 2015-06-07 00:00:00
Fetching games for 2015-06-08 00:00:00
Fetching games for 2015-06-09 00:00:00
Fetching games for 2015-0

In [315]:
def get_unique_game_dates(season):
    conn = sqlite3.connect("mlb.db")
    conn.row_factory = sqlite3.Row
    with conn:
        results = conn.execute(f"SELECT * FROM Games WHERE strftime('%Y', Day) = '{season}'").fetchall()
    conn.close()
    
    data = pd.DataFrame(results, columns=results[0].keys())
    # Dates have times at the end, so truncate to get ISO date
    return [x[:10] for x in data["Day"].unique()]

def insert_player_stats(season):
    game_dates = get_unique_game_dates(season)
    conn = sqlite3.connect("mlb.db")
    for date in game_dates:
        print(f"Fetching stats for {date}")
        data = get_player_stats(date)
        with conn:
            qmarks = str(tuple('?') * 69).replace("'", '')
            conn.executemany(f"INSERT OR REPLACE INTO PlayerStats VALUES {qmarks}", list(data.itertuples(index=False, name=None)))
        # Wait a half second before another API call
        time.sleep(0.5)
    conn.close()

In [316]:
seasons = range(2010, 2016)
for year in seasons:
    insert_player_stats(year)

Fetching stats for 2010-04-04
Fetching stats for 2010-04-05
Fetching stats for 2010-04-06
Fetching stats for 2010-04-07
Fetching stats for 2010-04-08
Fetching stats for 2010-04-09
Fetching stats for 2010-04-10
Fetching stats for 2010-04-11
Fetching stats for 2010-04-12
Fetching stats for 2010-04-13
Fetching stats for 2010-04-14
Fetching stats for 2010-04-15
Fetching stats for 2010-04-16
Fetching stats for 2010-04-17
Fetching stats for 2010-04-18
Fetching stats for 2010-04-19
Fetching stats for 2010-04-20
Fetching stats for 2010-04-21
Fetching stats for 2010-04-22
Fetching stats for 2010-04-23
Fetching stats for 2010-04-24
Fetching stats for 2010-04-25
Fetching stats for 2010-04-26
Fetching stats for 2010-04-27
Fetching stats for 2010-04-28
Fetching stats for 2010-04-29
Fetching stats for 2010-04-30
Fetching stats for 2010-05-01
Fetching stats for 2010-05-02
Fetching stats for 2010-05-03
Fetching stats for 2010-05-04
Fetching stats for 2010-05-05
Fetching stats for 2010-05-06
Fetching s

Fetching stats for 2011-06-12
Fetching stats for 2011-06-13
Fetching stats for 2011-06-14
Fetching stats for 2011-06-15
Fetching stats for 2011-06-16
Fetching stats for 2011-06-17
Fetching stats for 2011-06-18
Fetching stats for 2011-06-19
Fetching stats for 2011-06-20
Fetching stats for 2011-06-21
Fetching stats for 2011-06-22
Fetching stats for 2011-06-23
Fetching stats for 2011-06-24
Fetching stats for 2011-06-25
Fetching stats for 2011-06-26
Fetching stats for 2011-06-27
Fetching stats for 2011-06-28
Fetching stats for 2011-06-29
Fetching stats for 2011-06-30
Fetching stats for 2011-07-01
Fetching stats for 2011-07-02
Fetching stats for 2011-07-03
Fetching stats for 2011-07-04
Fetching stats for 2011-07-05
Fetching stats for 2011-07-06
Fetching stats for 2011-07-07
Fetching stats for 2011-07-08
Fetching stats for 2011-07-09
Fetching stats for 2011-07-10
Fetching stats for 2011-07-14
Fetching stats for 2011-07-15
Fetching stats for 2011-07-16
Fetching stats for 2011-07-17
Fetching s

Fetching stats for 2012-08-27
Fetching stats for 2012-08-28
Fetching stats for 2012-08-29
Fetching stats for 2012-08-30
Fetching stats for 2012-08-31
Fetching stats for 2012-09-01
Fetching stats for 2012-09-02
Fetching stats for 2012-09-03
Fetching stats for 2012-09-04
Fetching stats for 2012-09-05
Fetching stats for 2012-09-06
Fetching stats for 2012-09-07
Fetching stats for 2012-09-08
Fetching stats for 2012-09-09
Fetching stats for 2012-09-10
Fetching stats for 2012-09-11
Fetching stats for 2012-09-12
Fetching stats for 2012-09-13
Fetching stats for 2012-09-14
Fetching stats for 2012-09-15
Fetching stats for 2012-09-16
Fetching stats for 2012-09-17
Fetching stats for 2012-09-18
Fetching stats for 2012-09-19
Fetching stats for 2012-09-20
Fetching stats for 2012-09-21
Fetching stats for 2012-09-22
Fetching stats for 2012-09-23
Fetching stats for 2012-09-24
Fetching stats for 2012-09-25
Fetching stats for 2012-09-26
Fetching stats for 2012-09-27
Fetching stats for 2012-09-28
Fetching s

Fetching stats for 2014-04-08
Fetching stats for 2014-04-09
Fetching stats for 2014-04-10
Fetching stats for 2014-04-11
Fetching stats for 2014-04-12
Fetching stats for 2014-04-13
Fetching stats for 2014-04-14
Fetching stats for 2014-04-15
Fetching stats for 2014-04-16
Fetching stats for 2014-04-17
Fetching stats for 2014-04-18
Fetching stats for 2014-04-19
Fetching stats for 2014-04-20
Fetching stats for 2014-04-21
Fetching stats for 2014-04-22
Fetching stats for 2014-04-23
Fetching stats for 2014-04-24
Fetching stats for 2014-04-25
Fetching stats for 2014-04-26
Fetching stats for 2014-04-27
Fetching stats for 2014-04-28
Fetching stats for 2014-04-29
Fetching stats for 2014-04-30
Fetching stats for 2014-05-01
Fetching stats for 2014-05-02
Fetching stats for 2014-05-03
Fetching stats for 2014-05-04
Fetching stats for 2014-05-05
Fetching stats for 2014-05-06
Fetching stats for 2014-05-07
Fetching stats for 2014-05-08
Fetching stats for 2014-05-09
Fetching stats for 2014-05-10
Fetching s

Fetching stats for 2015-06-26
Fetching stats for 2015-06-27
Fetching stats for 2015-06-28
Fetching stats for 2015-06-29
Fetching stats for 2015-06-30
Fetching stats for 2015-07-01
Fetching stats for 2015-07-02
Fetching stats for 2015-07-03
Fetching stats for 2015-07-04
Fetching stats for 2015-07-05
Fetching stats for 2015-07-06
Fetching stats for 2015-07-07
Fetching stats for 2015-07-08
Fetching stats for 2015-07-09
Fetching stats for 2015-07-10
Fetching stats for 2015-07-11
Fetching stats for 2015-07-12
Fetching stats for 2015-07-14
Fetching stats for 2015-07-17
Fetching stats for 2015-07-18
Fetching stats for 2015-07-19
Fetching stats for 2015-07-20
Fetching stats for 2015-07-21
Fetching stats for 2015-07-22
Fetching stats for 2015-07-23
Fetching stats for 2015-07-24
Fetching stats for 2015-07-25
Fetching stats for 2015-07-26
Fetching stats for 2015-07-27
Fetching stats for 2015-07-28
Fetching stats for 2015-07-29
Fetching stats for 2015-07-30
Fetching stats for 2015-07-31
Fetching s